In [13]:
import spacy
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from statistics import mode
from urllib.request import urlopen
from bs4 import BeautifulSoup

nlp = spacy.blank("en")

In [29]:
TRAIN_DATA = [  ("India today is an article express.", {"entities":[(0,11,"ARTICLE")]}),
                ("Sushil Kumar is a convict", {"entities":[(0,12,"NAME")]}),
                ("Sushil Ajay is a convict", {"entities":[(0,6,"NAME"),(7,11,"NAME")]}),
                ("May 25, 2021 is when the crime was reported", {"entities":[(0,12,"DATE")]}),
                ("16:11 is the time", {"entities":[(0,5,"TIME")]}),
                ("Delhi is a location", {"entities":[(0,5,"LOCATION")]}),
                ("he murder Sagar", {"entities":[(3,9,"TYPE")]}),
                ("Ajay Kumar is a convict", {"entities":[(0,10,"NAME")]}),
                ("Hemanta Kumar Nath is a convict", {"entities":[(0,18,"NAME")]}),
                ("May 25, 2021", {"entities":[(0,12,"DATE")]}),
                ("03:15 is the time", {"entities":[(0,5,"TIME")]}),
                ("Guwahati is a location", {"entities":[(0,8,"LOCATION")]}),
                ("murder Sagar", {"entities":[(0,6,"TYPE")]}),
                ("May 17, 2021", {"entities":[(0,12,"DATE")]}),
                ("10:45 is the time", {"entities":[(0,5,"TIME")]}),
                ("delhi is a location", {"entities":[(0,5,"LOCATION")]}),
                ("murder", {"entities":[(0,6,"TYPE")]}),
                ("Vikas Dubey  is a convict", {"entities":[(0,11,"NAME")]}),
                ("june 09, 2020", {"entities":[(0,13,"DATE")]}),
                ("17:48 is the time", {"entities":[(0,5,"TIME")]}),
                ("kanpur is a location", {"entities":[(0,6,"LOCATION")]}),
                ("gangster Dubey", {"entities":[(0,14,"TYPE")]}),
                ("Vikrant Vir  is a convict", {"entities":[(0,11,"NAME")]}),
                ("september 09, 2020", {"entities":[(0,18,"DATE")]}),
                ("22:24 is the time", {"entities":[(0,5,"TIME")]}),
                ("Aligarh is a location", {"entities":[(0,7,"LOCATION")]}),
                ("rape vir", {"entities":[(0,4,"TYPE")]}),
                ("Nilesh  is a convict", {"entities":[(0,6,"NAME")]}),
                ("April 21, 2020", {"entities":[(0,14,"DATE")]}),
                ("18:57 is the time", {"entities":[(0,5,"TIME")]}),
                ("Mumbai is a location", {"entities":[(0,6,"LOCATION")]}),
                ("fraud vir", {"entities":[(0,5,"TYPE")]}),
                ("SI K Balakrishnan is a convict", {"entities":[(0,17,"NAME")]}),
                ("October 27, 2020", {"entities":[(0,16,"DATE")]}),
                ("09:15 is the time", {"entities":[(0,5,"TIME")]}),
                ("Sathankulam is a location", {"entities":[(0,11,"LOCATION")]}),
                ("assault vir", {"entities":[(0,7,"TYPE")]}),
                ("Tausif is a convict", {"entities":[(0,6,"NAME")]}),
                ("October 27, 2020", {"entities":[(0,16,"DATE")]}),
                ("16:37 is the time", {"entities":[(0,5,"TIME")]}),
                ("Faridabad is a location", {"entities":[(0,9,"LOCATION")]}),
                ("Murder vir", {"entities":[(0,6,"TYPE")]}),
                ("asianage is an article express.", {"entities":[(0,8,"ARTICLE")]}),
                ("Mandvi Hidma is a convict", {"entities":[(0,12,"NAME")]}),
                ("April 13, 2021", {"entities":[(0,14,"DATE")]}),
                ("09:58 is the time", {"entities":[(0,5,"TIME")]}),
                ("Chhattisgarh is a location", {"entities":[(0,12,"LOCATION")]}),
                ("Murder vir", {"entities":[(0,6,"TYPE")]}),
                ("deccanchronicle is an article express.", {"entities":[(0,15,"ARTICLE")]}),
                ("Narayana Reddy is a convict", {"entities":[(0,14,"NAME")]}),
                ("May 25, 2021", {"entities":[(0,12,"DATE")]}),
                ("11:12 is the time", {"entities":[(0,5,"TIME")]}),
                ("Vijayawada is a location", {"entities":[(0,10,"LOCATION")]}),
                ("Harassment vir", {"entities":[(0,10,"TYPE")]}),
                ("Ajaz Khan is a convict", {"entities":[(0,9,"NAME")]}),
                ("Mar 31, 2021", {"entities":[(0,12,"DATE")]}),
                ("11:00 is the time", {"entities":[(0,5,"TIME")]}),
                ("Mumbai is a location", {"entities":[(0,6,"LOCATION")]}),
                ("Drugs vir", {"entities":[(0,5,"TYPE")]}),
                ("Swati Laxman Malik is a convict", {"entities":[(0,18,"NAME")]}),
                ("March 12, 2021", {"entities":[(0,14,"DATE")]}),
                ("04:59 is the time", {"entities":[(0,5,"TIME")]}),
                ("Nagpur is a location", {"entities":[(0,6,"LOCATION")]}),
                ("Murder vir", {"entities":[(0,6,"TYPE")]}),
                ("Greaterkashmir", {"entities":[(0,14,"ARTICLE")]}),
                ("gang",{"entities":[(0,4,"NAME")]}),
                ("Company can also commit crime",{"entities":[(0,7,"NAME")]}),
                ("April 25, 2021", {"entities":[(0,14,"DATE")]}),
                ("1:00 AMApril", {"entities":[(0,4,"TIME")]}),
                ("Sherabad Khore Pattan", {"entities":[(0,14,"LOCATION")]}),
                ("bank robbery", {"entities":[(5,12,"TYPE")]}),
                ("TheHindu", {"entities":[(0,8,"ARTICLE")]}),
                ("Jai Kishan", {"entities":[(0,10,"NAME")]}),
                ("May 16, 2021", {"entities":[(0,12,"DATE")]}),
                ("23:01 IST", {"entities":[(0,5,"TIME")]}),
                ("NEW DELHI", {"entities":[(0,9,"LOCATION")]}),
                ("oxygen cylinder scam", {"entities":[(16,20,"TYPE")]}),
                ("Yashodha", {"entities":[(0,8,"NAME")]}),
                ("May 17, 2021", {"entities":[(0,12,"DATE")]}),
                ("22:41 IST", {"entities":[(0,5,"TIME")]}),
                ("Banglore", {"entities":[(0,8,"LOCATION")]}),
                ("Christopher Dias", {"entities":[(0,16,"NAME")]}),
                ("October 15, 2020", {"entities":[(0,16,"DATE")]}),
                ("9:55:42 am", {"entities":[(0,4,"TIME")]}),
                ("Mumbai", {"entities":[(0,6,"LOCATION")]}),
                ("murder", {"entities":[(0,6,"TYPE")]}),  
                ("Indian express", {"entities":[(0,14,"ARTICLE")]}),
                ("Hemraj Bharadwaj", {"entities":[(0,16,"NAME")]}),
                ("May 1, 2021", {"entities":[(0,11,"DATE")]}),
                ("Mandi district of Himachal Pradesh", {"entities":[(0,14,"LOCATION")]}),
                ("drug trafficking", {"entities":[(0,16,"TYPE")]}),
                ("NDTV", {"entities":[(0,4,"ARTICLE")]}),
                ("Kesh Kumari", {"entities":[(0,11,"NAME")]}),
                ("March 30, 2021", {"entities":[(0,14,"DATE")]}),
                ("3:46 pm", {"entities":[(0,4,"TIME")]}),
                ("Pure Baijnath village", {"entities":[(0,13,"LOCATION")]}),
                ("murder", {"entities":[(0,6,"TYPE")]}),
                ("Times now", {"entities":[(0,9,"ARTICLE")]}),
                ("Neeraj Chaube", {"entities":[(0,13,"NAME")]}),
                ("Mohammad Afsar", {"entities":[(0,14,"NAME")]}),
                ("May 26, 2021", {"entities":[(0,12,"DATE")]}),
                ("11:41 IST", {"entities":[(0,5,"TIME")]}),
                ("Noida", {"entities":[(0,5,"LOCATION")]}),
                ("extorting money", {"entities":[(0,15,"TYPE")]}),
                ("Manikandan", {"entities":[(0,10,"NAME")]}),
                ("May 08, 2021", {"entities":[(0,12,"DATE")]}),
                ("13:14 IST", {"entities":[(0,5,"TIME")]}),
                ("Mumbai", {"entities":[(0,6,"LOCATION")]}),
                ("theft case", {"entities":[(0,5,"TYPE")]})
             ]
                

In [4]:
import random
import spacy
from tqdm import tqdm
from spacy.training import Example

def train_spacy(data,iterations):
    n_iter = iterations
    nlp = spacy.blank('en') 
    
    if 'ner' not in nlp.pipe_names:       
        ner = nlp.add_pipe('ner', last=True)
    else:
        ner = nlp.get_pipe("ner")

    for _, annotations in data:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop = 0.2, sgd = optimizer, losses = losses)
            
        return nlp


model_nlp = train_spacy(TRAIN_DATA, 500)

E:\Anaconda\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "9:55:42 am" with entities "[(0, 4, 'TIME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [7]:
file = open("Articles/txt1.txt",encoding="utf-8").read()
text = file
print(text)

Sushil Kumar arrested: Sagar Rana's post mortem report suggests wrestler was hit by blunt object on head - Sports News 





 

 Jump to navigation
















  

NewsLIVE TV APPMagazine   Login Home MY FEED India  World  Business  Tech Movies  Sports  Science Health Videos  Corona Education Malayalam Fact Check  Visual Stories Law Quiz Gaming DIU NewsMo  Cities  Trending Binge Watch  Auto  Lifestyle  Television  AQI CODE OF ETHICS Rate Card Dark Mode       Speak Now Listening Re SpeakSubmitxNewsSportsOther SportsSushil Kumar arrested: Sagar Rana's post mortem report suggests wrestler was hit by blunt object on      

 Sushil Kumar arrested: Sagar Rana's post mortem report suggests wrestler was hit by blunt object on headWrestler Sagar Rana, who used to compete in the 97 kg Greco-Roman category, lost his life in a scuffle involving two groups inside the Chhatrasal Stadiumadvertisement

  India Today Web Desk    New DelhiMay 25, 2021UPDATED: May 25, 2021 16:11 IST  Sushil Kumar was

In [8]:
doc = model_nlp(text)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [1]:
df_type = []
df_loc = [] 
df_name = []
df_gender = []
df_time = []
df_date = []
df_article = []
df_link = []

t_type = []
t_loc = [] 
t_name = []
t_gender = []
t_time = []
t_date = []
t_article = []

tokens=[]

In [186]:
import statistics
from statistics import mode

Type = ["murder","gangster","rape","fraud","attack","drugs","harrassment","theft","terrorism","more","cyber crime","smuggle","extortion","kidnap"]

for entities in doc.ents:   
    if(entities.label_ == "ARTICLE"):
        t_article.append(entities.text)
        
    if(entities.label_ == "TYPE"):
        if(entities.text in Type):
            t_type.append(entities.text)  
            
    if(entities.label_ == "LOCATION"):
        t_loc.append(entities.text)
        
    if(entities.label_ == "NAME"):
        t_name.append(entities.text)
        
    if(entities.label_ == "TIME"):
        if(re.match("^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$",entities.text)):
            t_time.append(entities.text)
            
    if(entities.label_ == "DATE"):
        if(re.match("^[a-zA-Z]{3,}\s[0-9]{2,}[,]\s*[0-9]{2,}$",entities.text)):
            t_date.append(entities.text)
            
print(t_article,"\n",t_type,"\n",t_name,"\n",t_gender,"\n",t_date,"\n",t_time,"\n",t_loc,"\n") 
 
df_article.append(mode(t_article))
df_type.append(mode(t_type))
df_name.append(mode(t_name))
df_date.append(t_date[0])
df_time.append(t_time[0])
df_loc.append(mode(t_loc))

tokens = word_tokenize(text)
for x in tokens:
    if(x=="he" or x=="male"):
        t_gender.append("m")
    elif(x=="she" or x=="women"):
        t_gender.append("f")
    elif(x=="gang" or x=="they"):
        t_gender.append("gang")
    elif(x=="company"):
        t_gender.append("org")
    elif(x=="child" or x=="minor"):
        t_gender.append("minor")
        
df_gender.append(mode(t_gender))

           
print(df_article,df_type,df_name,df_gender,df_date,df_time,df_loc,df_link)

['India  ', 'India Today', 'In the', ', who', 'Trauma Center', 'treatment', 'Scars are', 'Tuesday and', 'India Today', 'Apple announces', ', but', 'Good news', 'Treatment Protocol', 'India  ', 'TodayCosmopolitanIndia Today', 'Ishq FMEducation', 'India Today', 'AajtakIndia Today', 'India 2018Sahitya', 'Thomson PressWelfare'] 
 ['murder'] 
 ['Sushil Kumar', 'Sagar Rana', 'Sports News', 'Jump', 'TV', 'APPMagazine', 'Tech Movies', 'Sports', 'Health Videos', 'Corona Education Malayalam', 'Fact Check', 'Law Quiz Gaming', 'Trending Binge Watch', 'Auto', 'Television', 'Rate Card Dark Mode', 'Speak Now Listening', 'SpeakSubmitxNewsSportsOther', 'SportsSushil Kumar', 'Sagar Rana', 'Sushil Kumar', 'Sagar Rana', 'Sagar Rana,', 'Greco', '-Roman', 'Chhatrasal Stadiumadvertisement', 'Web Desk', 'New DelhiMay', ', 2021UPDATED', 'Sushil Kumar', 'Crime Branch', 'Chhatrasal Stadium', 'The Babu Jagjivan Ram Memorial', 'Hospital', 'Greco', '-Roman', 'Crime Branch', 'Sushil Kumar', 'Chhatrasal Stadium', 'Ra

In [38]:
import spacy
nlp = model_nlp
examples = []
data = [("A body of a rape victim was found by Maharashta police. The accused was identified as Raj Gopal who surrended himself to the Mumbai police deprtment. The crime was reported in Ndtv news on May 2, 2021 2:30 am.", {"entities": [(37, 47, 'LOCATION'),(12, 16, 'TYPE'),(86,95,"NAME"),(201,205,"TIME"),(189,200,"DATE")]})]
for text, annots in data:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))
print(nlp.evaluate(examples))

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.5, 'ents_r': 0.6, 'ents_f': 0.5454545454545454, 'ents_per_type': {'LOCATION': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'NAME': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'TYPE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'TIME': {'p': 1.0, 'r': 1.0, 'f': 1.0}}, 'speed': 6893.014010343441}


In [39]:
file1 = open("Articles/6.txt",encoding="utf-8").read()
text1 = file1
doc1 = model_nlp(text1)
spacy.displacy.render(doc1, style='ent',jupyter=True)

In [ ]:
Type = ["murder","gangster","rape","fraud","attack","drugs","harrassment","theft","terrorism","tortured", "more","cyber crime","smuggle","extortion","kidnap"]

#change the url to scrape different e-articles.
url = "https://www.indiatoday.in/sports/other-sports/story/wrestler-sushil-kumar-sushil-kumar-wrestler-sushil-kumar-case-sushil-kumar-arrested-sushil-kumar-arrest-sushil-kumar-murder-case-sushil-kumar-murder-sushil-kuma-1806782-2021-05-25"
df_link.append(url)
html = urlopen(url).read()
soup = BeautifulSoup(html, "html.parser")

for script in soup(["script", "style"]): 
    script.extract()  

text = soup.get_text()
text = text.encode('utf-8')

file = open("Articles/txt1.txt","wb") #change the text file for each url
file.write(text)
file.close()

for entities in doc.ents:   
    if(entities.label_ == "ARTICLE"):
        t_article.append(entities.text)
        
    if(entities.label_ == "TYPE"):
        if(entities.text in Type):
            t_type.append(entities.text)  
            
    if(entities.label_ == "LOCATION"):
        t_loc.append(entities.text)
        
    if(entities.label_ == "NAME"):
        t_name.append(entities.text)
        
    if(entities.label_ == "TIME"):
        if(re.match("^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$",entities.text)):
            t_time.append(entities.text)
            
    if(entities.label_ == "DATE"):
        if(re.match("^[a-zA-Z]{3,}\s[0-9]{2,}[,]\s*[0-9]{2,}$",entities.text)):
            t_date.append(entities.text)
            
print(t_article,"\n",t_type,"\n",t_name,"\n",t_gender,"\n",t_date,"\n",t_time,"\n",t_loc,"\n") 

for x in texts:
    file1 = open(x,encoding="utf-8").read()
    text1 = file1
    doc1 = model_nlp(text1)
    spacy.displacy.render(doc1, style='ent',jupyter=True)
    
    for entities in doc1.ents:   
        if(entities.label_ == "ARTICLE"):
            t_article.append(entities.text)

        if(entities.label_ == "TYPE"):
            t_type.append(entities.text)  

        if(entities.label_ == "LOCATION"):
            t_loc.append(entities.text)

        if(entities.label_ == "NAME"):
            t_name.append(entities.text)

        if(entities.label_ == "TIME"):
            if(re.match("^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$",entities.text)):
                t_time.append(entities.text)

        if(entities.label_ == "DATE"):
            if(re.match("^[a-zA-Z]{3,}\s[0-9]{2,}[,]\s*[0-9]{2,}$",entities.text)):
                t_date.append(entities.text)

    if ((not len(t_article)) and (not len(t_type)) and (not len(t_name)) and (not len(t_date)) and (not len(t_time)) and (not len(d_loc))):
        df_article.append(mode(t_article))
        df_type.append(mode(t_type))
        df_name.append(mode(t_name))
        df_date.append(t_date[0])
        df_time.append(t_time[0])
        df_loc.append(mode(t_loc))


    tokens = word_tokenize(text)
    for x in tokens:
        if(x=="he" or x=="male"):
            t_gender.append("m")
        elif(x=="she" or x=="women"):
            t_gender.append("f")
        elif(x=="gang" or x=="they"):
            t_gender.append("gang")
        elif(x=="company"):
            t_gender.append("org")
        elif(x=="child" or x=="minor"):
            t_gender.append("minor")

    df_gender.append(mode(t_gender))

    t_type = []
    t_loc = [] 
    t_name = []
    t_gender = []
    t_time = []
    t_date = []
    t_article = []

In [8]:
table1 = [df_article,df_type,df_name,df_gender,df_date,df_time,df_loc,df_link]
print(table1)

     Sno Article_name      Type       Criminal_name Criminal_gender  \
0      1   Indiatoday    murder  Hemanth kumar nath               m   
1      2      Indiatv    murder            Hoti lal               m   
2      3   Indiatoday  gangster         Vikas Dubey               m   
3      4   Indiatoday      rape         Vikrant vir               m   
4      5   Indiatoday     fraud              Nilesh               m   
..   ...          ...       ...                 ...             ...   
117  118         Ndtv     fraud      Vaibhav sharma               m   
118  119         Ndtv    murder                gang            gang   
119  120         Ndtv        1+                   -               m   
120  121         Ndtv        1+        Gooty rajesg               m   
121  122         Ndtv    murder               Bittu               m   

     Reported_date Reported_time   Location  \
0     25 May, 2021          3:15   Guwahati   
1     17 May, 2021         10:45  New Delhi   
2    0